In [0]:
%sh apt-get update && apt-get install -y gdal-bin libgdal-dev postgis

In [0]:
%sh
export PLUGIN_DIR=$(gdal-config --plugindir)
export CATALOG=catalog_stuarlqxck_tcgppf
export SCHEMA=raster
export PLUGIN_VOLUME=ecw

cp /Volumes/$CATALOG/$SCHEMA/$PLUGIN_VOLUME/gdal_ECW_JP2ECW.so $PLUGIN_DIR

In [0]:
%sh
export CPL_DEBUG=ON
gdalinfo --formats

In [0]:
%sh which raster2pgsql

In [0]:
%sh raster2pgsql

In [0]:
%sh raster2pgsql -G

In [0]:
import os
import subprocess
import glob

# Local temp directory on the driver
local_dir = "/tmp/georaster_demo"
os.makedirs(local_dir, exist_ok=True)

srid = "27700"                 # adjust if using a different GeoTIFF
catalog = "catalog_stuarlqxck_tcgppf"
lakehouse_schema = "raster"
imagery_volume = "imagery"

lakebase_schema = "raster"            # your Lakebase schema
table = "imagery"       # table to create
output_sql = os.path.join(local_dir, f"{table}.sql")

rasters = glob.glob(f"/Volumes/{catalog}/{lakehouse_schema}/{imagery_volume}/*.ecw", recursive=True)

In [0]:
rasters[0]

In [0]:
%sh gdalinfo /Volumes/catalog_stuarlqxck_tcgppf/raster/imagery/Ortho_IRRGB_P00064335_20140216_20140216_20cm_res.ecw

In [0]:
# Example tiling and options – change as desired
# -I: create GiST index, -C: apply raster constraints
# -M: vacuum analyze, -F: add a raster column named "rast"
# -t 100x100: tile size
cmd = [
    "raster2pgsql",
    "-I",
    "-C",
    "-d",
    "-e",
    "-M",
    "-F",
    "-l", "2,4,8",
    "-t", "512x512",
    "-s", srid,
    *rasters,
    f"{lakebase_schema}.{table}"
]

print("Running:", " ".join(cmd))

with open(output_sql, "w") as f:
    subprocess.run(cmd, check=True, stdout=f)

print("Generated SQL at:", output_sql)

In [0]:
%pip install databricks-sdk==0.90.0 --force-reinstall
%restart_python

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.postgres import RequestedClaims, RequestedResource

w = WorkspaceClient()
project_id = "61ad7f1c-95f4-4143-8b66-000942afcb89"
branch_id = "br-odd-silence-e1b8my4j"
endpoint_id = "ep-blue-mouse-e1jj0xrm"
endpoint = f"projects/{project_id}/branches/{branch_id}/endpoints/{endpoint_id}"
creds = w.postgres.generate_database_credential(endpoint)

In [0]:
os.environ["PGPASSWORD"] = creds.token

+ create database raster
+ create schema raster
+ install postgis and postgis_raster extensions
CREATE INDEX ix_rasters_rast ON imagery USING gist(ST_ConvexHull(rast)); (this is in the script that is creataed by raster2pgsql)
SELECT AddRasterConstraints('imagery'::name, 'rast'::name); (this is in the script that is creataed by raster2pgsql)
SELECT ST_CreateOverview('raster.imagery'::regclass, 'rast', 2);

In [0]:
%sh psql -h 'ep-blue-mouse-e1jj0xrm.database.eastus2.azuredatabricks.net' --set=sslmode=require -d 'raster' -U 'stuart.lynn@databricks.com' -f /tmp/georaster_demo/imagery.sql